# Multi-agent Gathering Environment

In this notebook, we will get acquainted with the multi-agent Gathering game environment created by HumanCompatibleAI on GitHub.

In [1]:
import random
import time
import platform

from env import GatheringEnv   # This is the Game Environment
from model import Policy   # Use the Policy defined below instead

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

print("Python version: ", platform.python_version())

Python version:  3.6.4


## Policy Gradient Agent

The agent's policy is a simple 2-layer NN. The current code only allows one agent to be trained in a Gathering environment with the other agents being random agents.

Note that the weights are saved and loaded from a fixed single file.

In [167]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class Policy(nn.Module):
    
    # an index parameter may be needed to allow for multiple learning agents
    def __init__(self, observation_size):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(observation_size, 128)
        self.affine2 = nn.Linear(128, 8)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.affine2(x)
        return F.softmax(action_scores)

    # The weights should be allowed to be saved into and loaded different model files
    def save_weights(self):
        torch.save(self.state_dict(), 'model.pkl')   

    def load_weights(self):
        self.load_state_dict(torch.load('model.pkl'))

    def select_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self(Variable(state))
        action = probs.multinomial()
        self.saved_actions.append(action)
        return action.data[0, 0]

## Maps

GatheringEnv will render a Gathering map based on text files. The following are some of the available text files:

* default.txt  
* open.txt  
* region_multi_entrance.txt  
* region_no_walls.txt  
* region_single_entrance.txt  
* region_unequal_single_entrance.txt  
* small.txt

The default map is the map utilized by DeepMind's paper.

The current code has several limitations:
* N_agents must be 2
* The 2nd agent can only be random

In [4]:
env = GatheringEnv(n_agents=2, map_name='default')  # Change map here; n must be 2 or under

# Env API is similar to that of OpenAI Gym
state_n = env.reset()
env.render()

# Load previously trained policy agent
policy = Policy(env.state_size)
policy.load_weights()

n_tags = 0 # count number of times learning agent tag the random agent
n_steps = 300

# Render for 1000 steps
for _ in range(n_steps):
    action = policy.select_action(state_n[0])
    state_n, reward_n, done_n, info_n = env.step([
        action,                           # Agent 1 is policy AI
        random.randrange(0, 8)            # Agent 2 is random
    ])
    
    if action is 6:
        n_tags += 1
        
    if any(done_n):
        break
    env.render()
    time.sleep(1/30)  # Change speed of video rendering

env.close()  # Close the rendering window
print ("Aggressiveness is {:.2f}".format(n_tags/n_steps))

Aggressiveness is 0.05


## REINFORCE

The single agent is then trained with the basic policy gradient algorithm REINFORCE.

We discard the model.pkl and the hyperparameters provided by HumanCompatibleAI. Using lr = 1e-3 after 1000 training episodes, the agent learns how to collect apples and occasionally laser-tag the random agent:

SA_ep1000_lr1e-3_default_r390.pkl

After 3000 episodes, agent performs marginally better. It still does not know how to tag the random agent consistently.

SA_ep3000_lr1e-3_default_r440.pkl

Aggressiveness is only 0.03. Meaning the learning agent is tagging only 3 out of every 100 steps, when the random agent is much more aggressively tagging 1 out of every 8 steps.

In [39]:
# Adapted from https://github.com/pytorch/examples/blob/2dca104/reinforcement_learning/reinforce.py
# Licensed under BSD 3-clause: https://github.com/pytorch/examples/blob/2dca10404443ce3178343c07ba6e22af13efb006/LICENSE

import random
from itertools import count

from env import GatheringEnv
# from model import Policy   Will instead use the Model in the Notebook

import numpy as np

import torch
import torch.autograd as autograd
import torch.optim as optim

# We remove the argparse code and replace with hard parameters
gamma = 0.99
seed = 543
render=False
log_interval=20
target_reward = 500
                 
# Start the Gathering environment
env = GatheringEnv(n_agents=2, map_name='default')
env.seed(seed)
torch.manual_seed(seed)

# Load the learning agent (single)
policy = Policy(env.state_size)

# policy.load_weights()     Start training from scratch
optimizer = optim.Adam(policy.parameters(), lr=1e-3)   # Original lr = 1e-2

# This is the task list to finish a game episode
def finish_episode():
    """ 
    Based on REINFORCE, policy gradient is calculated at the end of an episode.
    It is then used to update the Policy's weights
    """
    
    # Note that the current code only accommodate 1 learning agent.
    # Reward for each time step is stored in the list policy.rewards[] --> r(t)
    R = 0  # noqa
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R  # noqa
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + np.finfo(np.float32).eps)
    
    # Calculate policy gradient Σ_t[r(t) ∇_θ log(π_θ(s_t,a_t))
    for action, r in zip(policy.saved_actions, rewards):
        action.reinforce(r)     # Note that action.reinforce has been deprecated since torch V0.3
    optimizer.zero_grad()
    autograd.backward(policy.saved_actions, [None for _ in policy.saved_actions])
    optimizer.step()
    
    del policy.rewards[:]
    del policy.saved_actions[:]


running_reward = None
best_reward = None  # Keep track of best running average for storing best_model
for i_episode in range(1,3001):   # For each game episode
    
    # Some basic initialization
    state = env.reset()[0]
    episode_reward = 0
    
    for t in range(1000):  # Don't infinite loop while learning
        
        # 1. learning agent selects an action
        action = policy.select_action(state)
        
        # 2. environment accepts action from learning agent and random action from 2nd agent
        # and doles out next state, reward and if the episode is done
        state_n, reward_n, done_n, _ = env.step([action, random.randrange(0, 8)])
        state = state_n[0]
        reward = reward_n[0]
        done = done_n[0]
        
        if render:   # render learning if set to True
            env.render()
            
        # Store rewards into array for REINFORCE    
        policy.rewards.append(reward)
        episode_reward += reward
        
        if done:
            break

    # Update reward statistics
    if running_reward is None:
        running_reward = episode_reward
    running_reward = running_reward * 0.99 + episode_reward * 0.01
    
    finish_episode()    # Finish up game episode
        
    if i_episode % log_interval == 0:
        print('Episode {}\tLast reward: {:5d}\tAverage reward: {:.2f}'.format(
            i_episode, episode_reward, running_reward))
        # Only save NN weights if running_reward is better than best_reward
        if best_reward is None:
            best_reward = running_reward
            policy.save_weights()   # Save NN weights if this is the first save
        else:
            if running_reward > best_reward:
                policy.save_weights()   # Save NN weights only if agent performs better
        
    if running_reward > target_reward:    # Rather weird to 
        print("Solved! Running reward is now {} and "
              "the last episode received {} reward!".format(running_reward, episode_reward))
        policy.save_weights()   # Save NN weights
        env.close()  # Close the rendering window
        break

Episode 20	Last reward:     0	Average reward: 0.20
Episode 40	Last reward:     3	Average reward: 0.94
Episode 60	Last reward:     0	Average reward: 1.17
Episode 80	Last reward:     1	Average reward: 1.51
Episode 100	Last reward:     0	Average reward: 2.43
Episode 120	Last reward:    35	Average reward: 5.17
Episode 140	Last reward:    51	Average reward: 10.74
Episode 160	Last reward:    59	Average reward: 16.84
Episode 180	Last reward:   108	Average reward: 30.03
Episode 200	Last reward:   161	Average reward: 38.59
Episode 220	Last reward:   193	Average reward: 53.23
Episode 240	Last reward:   181	Average reward: 71.10
Episode 260	Last reward:   175	Average reward: 88.67
Episode 280	Last reward:   205	Average reward: 106.47
Episode 300	Last reward:   196	Average reward: 124.59
Episode 320	Last reward:   227	Average reward: 138.81
Episode 340	Last reward:   251	Average reward: 155.71
Episode 360	Last reward:   240	Average reward: 173.52
Episode 380	Last reward:   230	Average reward: 187.

## Understand the Gathering Environment

To be able to implement multiple learning agents in Gathering, we need to understand how the Class GatheringEnv() works.

The environment is built on top of OpenAI Gym. The coding is a bit of a hack, and require a lot of documentation in order to understand how it works.

In [151]:
import collections
import itertools
import os.path
import tkinter as tk

import gym
import gym.envs.registration
import gym.spaces

import numpy as np

# The 8 actions
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3
ROTATE_RIGHT = 4
ROTATE_LEFT = 5
LASER = 6
NOOP = 7


class GatheringEnv(gym.Env):

    # Some basic parameters for the Gathering Game
    metadata = {'render.modes': ['human']}
    scale = 20           # Used to scale to display during rendering

    # Viewbox is to implement partial observable Markov game
    viewbox_width = 10
    viewbox_depth = 20
    padding = max(viewbox_width // 2, viewbox_depth - 1)  # essentially 20-1=19

    # To help agents distinquish between themselves, the other agents and the apple
    agent_colors = ['red', 'yellow']

    # A function to build the game space from a text file
    def _text_to_map(self, text):

        m = [list(row) for row in text.splitlines()]  # regard "\r", "\n", and "\r\n" as line boundaries 
        l = len(m[0])
        for row in m:   # Check for errors in text file
            if len(row) != l:
                raise ValueError('the rows in the map are not all the same length')

        # This essentially add a padding of 20 cells around the region enclosed by the walls or by the
        # food (if there is no wall). During rendering you will observe this padded region when a laser
        # is fired     
        def pad(a):
            return np.pad(a, self.padding + 1, 'constant')

        a = np.array(m).T
        self.initial_food = pad(a == 'O').astype(np.int)    # Pad 20 around food
        self.walls = pad(a == '#').astype(np.int)           # Pad 20 around the walls


    # This is run when the environment is created
    def __init__(self, n_agents=1, map_name='default'):

        self.n_agents = n_agents    # Set number of agents
        self.root = None            # For rendering

        # Create game space from text file
        if not os.path.exists(map_name):
            expanded = os.path.join('maps', map_name + '.txt')
            if not os.path.exists(expanded):
                raise ValueError('map not found: ' + map_name)
            map_name = expanded
        with open(map_name) as f:
            self._text_to_map(f.read().strip())    # This sets up self.initial_food and self.walls

        # Populate the rest of environment parameters
        self.width = self.initial_food.shape[0]
        self.height = self.initial_food.shape[1]

        # This is a partial observable Markov game. So the state for each agent is a single frame of the  
        # 10x20 of RGB pixels (view box). There is no stacked frames.
        self.state_size = self.viewbox_width * self.viewbox_depth * 4
        self.observation_space = gym.spaces.MultiDiscrete([[[0, 1]] * self.state_size] * n_agents)
        self.action_space = gym.spaces.MultiDiscrete([[0, 7]] * n_agents)   # Action space for n agents

        self._spec = gym.envs.registration.EnvSpec(**_spec)
        self.reset()    # Reset environment
        self.done = False

    # A function to take the game one step forward
    # Inputs: a list of actions indexed by agent
    def _step(self, action_n):

        assert len(action_n) == self.n_agents  # Error check for action list
        # Set action of tagged agents to NOOP
        action_n = [NOOP if self.tagged[i] else a for i, a in enumerate(action_n)]

        # Initialize variables for movement and for beam
        self.beams[:] = 0
        movement_n = [(0, 0) for a in action_n]

        # Update movement if action is UP, DOWN, RIGHT or LEFT
        for i, (a, orientation) in enumerate(zip(action_n, self.orientations)):
            if a not in [UP, DOWN, LEFT, RIGHT]:
                continue
            # a is relative to the agent's orientation, so add the orientation
            # before interpreting in the global coordinate system.
            #
            # This line is really not obvious to read. Replace it with something
            # clearer if you have a better idea.
            a = (a + orientation) % 4
            movement_n[i] = [
                (0, -1),  # up/forward
                (1, 0),   # right
                (0, 1),   # down/backward
                (-1, 0),  # left
            ][a]

        # The code section below updates agent location based on actions that are movements    
        next_locations = [a for a in self.agents]  # Initialize next_locations
        # If a key is not found in the dictionary, then instead of a KeyError being thrown, a new entry 
        # is created.
        next_locations_map = collections.defaultdict(list)


        for i, ((dx, dy), (x, y)) in enumerate(zip(movement_n, self.agents)):  # For each agent
            if self.tagged[i]:
                continue        # skip agents that are tagged
            next_ = ((x + dx), (y + dy))   # Update next location
            if self.walls[next_]:
                next_ = (x, y)              # Do not move beyond wall
            next_locations[i] = next_
            next_locations_map[next_].append(i)   # Add agent number to next_location_map (???)

        # If there are more than 1 agent in the same location
        for overlappers in next_locations_map.values():
            if len(overlappers) > 1:
                for i in overlappers:
                    next_locations[i] = self.agents[i]  # return agent to their previous location
        self.agents = next_locations_map    # Update agent locations

        # If action is ROTATE_RIGHT, ROTATE_LEFT or LASER
        for i, act in enumerate(action_n):
            if act == ROTATE_RIGHT:
                self.orientations[i] = (self.orientations[i] + 1) % 4
            elif act == ROTATE_LEFT:
                self.orientations[i] = (self.orientations[i] - 1) % 4
            elif act == LASER:
                # Laser beam is 5x20 along the orientation of the agent
                self.beams[self._viewbox_slice(i, 5, 20, offset=1)] = 1   

        # Prepare obs_n, reward_n, done_n and info_n to be returned        
        obs_n = self.state_n    # obs_n is self.state_n
        reward_n = [0 for _ in range(self.n_agents)]
        done_n = [self.done] * self.n_agents
        info_n = [{}] * self.n_agents

        # This is really shitty code writing. If agent lands on a food cell, that cell is set to -15.
        # Then for each subsequent step, it is incremented by 1 until it reaches 1 again.
        # self.initial_food is the game space created from the text file whereby the cell with food 
        # is given the value of 1, every other cell has the value of 0.
        self.food = (self.food + self.initial_food).clip(max=1)

        # In case the agent lands on a cell with food, or is tagged
        for i, a in enumerate(self.agents):
            if self.tagged[i]:
                continue
            if self.food[a] == 1:
                self.food[a] = -15    # Food is respawned every 15 steps once it has been consumed
                reward_n[i] = 1       # Agent is given reward of 1
            if self.beams[a]:
                self.tagged[i] = 25   # If agent is tagged, it is removed from the game for 25 steps

        # Respawn agent after 25 steps
        for i, tag in enumerate(self.tagged):
            if tag == 1:
                # Relocate to a respawn point.
                for spawn_point in self.spawn_points:
                    if spawn_point not in self.agents:
                        self.agents[i] = spawn_point
                        break

        # This is where the tagged counter (25 steps) is updated
        self.tagged = [max(i - 1, 0) for i in self.tagged]   

        return obs_n, reward_n, done_n, info_n


    # Generate slice(tuple) to slice out observation space for agents
    def _viewbox_slice(self, agent_index, width, depth, offset=0):
        
        # These are inputs for generating an observation space for the agent
        left = width // 2
        right = left if width % 2 == 0 else left + 1
        x, y = self.agents[agent_index]

        # This is really hard-to-read code. Essentially, it generates the observation
        # spaces for an agent in all 4 orientations, then only return the one indexed
        # by its current orientation.
        # Note: itertools.starmap maps the orientation-indexed tuple to slice()
        return tuple(itertools.starmap(slice, (
            ((x - left, x + right), (y - offset, y - offset - depth, -1)),      # up
            ((x + offset, x + offset + depth), (y - left, y + right)),          # right
            ((x + left, x - right, -1), (y + offset, y + offset + depth)),      # down
            ((x - offset, x - offset - depth, -1), (y + left, y - right, -1)),  # left
        )[self.orientations[agent_index]]))


    # state_n (next state) is a property object. So this function is run everytime state_n is
    # called as a varaiable.
    @property
    def state_n(self):

        # Create a game space for agent locating
        agents = np.zeros_like(self.food)  

        # self.agent is a list of agent locations indexed by agent index
        for i, loc in enumerate(self.agents):    
            if not self.tagged[i]:
                agents[loc] = 1     # Mark the agent's location

        food = self.food.clip(min=0)   # Mark the food's location

        # Zero out next states for the agents
        s = np.zeros((self.n_agents, self.viewbox_width, self.viewbox_depth, 4))

        # Enumerate index, (agent orientation, agent location) by agent index
        for i, (orientation, (x, y)) in enumerate(zip(self.orientations, self.agents)):

            if self.tagged[i]:
                continue     # Skip if agent has been tagged out of the game

            # If agent is not tagged, ....

            # Construct the full state for the game, which consists of:
            # 1. Location of Food
            # 2. ???
            # 3. Location of agents
            # 4. Location of the walls
            full_state = np.stack([food, np.zeros_like(food), agents, self.walls], axis=-1)
            full_state[x, y, 2] = 0   # Zero out the agent's location ???

            # Create observation space for learning agent using _viewbox_slice()
            xs, ys = self._viewbox_slice(i, self.viewbox_width, self.viewbox_depth)
            observation = full_state[xs, ys, :]

            # Orient the observation space correctly
            s[i] = observation if orientation in [UP, DOWN] else observation.transpose(1, 0, 2)

        return s.reshape((self.n_agents, self.state_size))  # Return the agents' observations


    # To reset the environment
    def _reset(self):

        # Build food stash
        self.food = self.initial_food.copy()


        # Rebuild the wall (by subtracting padding from self.walls - very weird implementation!!!)
        # Essentially, think of a much larger game area (+20 cells on each side) surrounding the 
        # walled region.
        p = self.padding
        self.walls[p:-p, p] = 1
        self.walls[p:-p, -p - 1] = 1
        self.walls[p, p:-p] = 1
        self.walls[-p - 1, p:-p] = 1

        self.beams = np.zeros_like(self.food)  # self.beams region is as big as self.food (weird!)

        # Set up agent parameters
        # The agents are spawned at the right upper corner of the game area, one next to the other
        self.agents = [(i + self.padding + 1, self.padding + 1) for i in range(self.n_agents)]
        self.spawn_points = list(self.agents)
        self.orientations = [UP for _ in self.agents]   # Orientation = UP
        self.tagged = [0 for _ in self.agents]          # Tagged = False

        return self.state_n  # Since state_n is a property object, so it will call function _state_n()


    # To close the rendering window
    def _close_view(self):
        # If rendering window is active, close it
        if self.root:
            self.root.destroy()
            self.root = None
            self.canvas = None
        self.done = True   # The episode is done
    

    # TO render the game    
    def _render(self, mode='human', close=False):
        if close:
            self._close_view()
            return

        canvas_width = self.width * self.scale
        canvas_height = self.height * self.scale

        if self.root is None:
            self.root = tk.Tk()
            self.root.title('Gathering')
            self.root.protocol('WM_DELETE_WINDOW', self._close_view)
            self.canvas = tk.Canvas(self.root, width=canvas_width, height=canvas_height)
            self.canvas.pack()

        self.canvas.delete(tk.ALL)
        self.canvas.create_rectangle(0, 0, canvas_width, canvas_height, fill='black')

        def fill_cell(x, y, color):
            self.canvas.create_rectangle(
                x * self.scale,
                y * self.scale,
                (x + 1) * self.scale,
                (y + 1) * self.scale,
                fill=color,
            )

        for x in range(self.width):
            for y in range(self.height):
                if self.beams[x, y] == 1:
                    fill_cell(x, y, 'yellow')
                if self.food[x, y] == 1:
                    fill_cell(x, y, 'green')
                if self.walls[x, y] == 1:
                    fill_cell(x, y, 'grey')

        for i, (x, y) in enumerate(self.agents):
            if not self.tagged[i]:
                fill_cell(x, y, self.agent_colors[i])

        if False:
            # Debug view: see the first player's viewbox perspective.
            p1_state = self.state_n[0].reshape(self.viewbox_width, self.viewbox_depth, 4)
            for x in range(self.viewbox_width):
                for y in range(self.viewbox_depth):
                    food, me, other, wall = p1_state[x, y]
                    assert sum((food, me, other, wall)) <= 1
                    y_ = self.viewbox_depth - y - 1
                    if food:
                        fill_cell(x, y_, 'green')
                    elif me:
                        fill_cell(x, y_, 'cyan')
                    elif other:
                        fill_cell(x, y_, 'red')
                    elif wall:
                        fill_cell(x, y_, 'gray')
            self.canvas.create_rectangle(
                0,
                0,
                self.viewbox_width * self.scale,
                self.viewbox_depth * self.scale,
                outline='blue',
            )

        self.root.update()


    # To close the environment
    def _close(self):
        self._close_view()

    # To delete the environment
    def __del__(self):
        self.close()


_spec = {
    'id': 'Gathering-Luke-v001',
    'entry_point': GatheringEnv,
    'reward_threshold': 500,   # The environment threshold at 100 appears to be too low
}


gym.envs.registration.register(**_spec)


### Figuring out data structures

In [152]:
# Start the Gathering environment
env = GatheringEnv(n_agents=2, map_name='small')
np.set_printoptions(threshold=np.inf)

print (env.food.shape)
print (env.agents)
print (env.orientations)
for i,a in enumerate(env.agents):
    print (i, a)
    
for i, (orientation, (x, y)) in enumerate(zip(env.orientations, env.agents)):
    print (i, (orientation, (x, y)))

(64, 47)
[(20, 20), (21, 20)]
[0, 0]
0 (20, 20)
1 (21, 20)
0 (0, (20, 20))
1 (0, (21, 20))


### Figure out array manipulations

In [ ]:
[[[0, 1]] * 3] * 2

In [157]:
a = [[1,1,1]]
b = [[2,2,2]]
c = [[3,3,3]]
d = [[4,4,4]]

full_state = np.stack([a,b,c,d], axis=-1)
print (full_state)

full_state[:,1,:]=0
print (full_state)

full_state[:,:,2]=0
print (full_state)

[[[1 2 3 4]
  [1 2 3 4]
  [1 2 3 4]]]
[[[1 2 3 4]
  [0 0 0 0]
  [1 2 3 4]]]
[[[1 2 0 4]
  [0 0 0 0]
  [1 2 0 4]]]


### Figuring out _view_slice()

In [158]:
width = 10
depth = 15
offset = 0
x = 15
y = 20

# These are inputs for generating an observation space for the agent
left = width // 2
right = left if width % 2 == 0 else left + 1

for orient in range(4):
    viewbox = tuple(itertools.starmap(slice, (
            ((x - left, x + right), (y - offset, y - offset - depth, -1)),      # up
            ((x + offset, x + offset + depth), (y - left, y + right)),          # right
            ((x + left, x - right, -1), (y + offset, y + offset + depth)),      # down
            ((x - offset, x - offset - depth, -1), (y + left, y - right, -1)),
        )[orient]))

    print (viewbox)

(slice(10, 20, None), slice(20, 5, -1))
(slice(15, 30, None), slice(15, 25, None))
(slice(20, 10, -1), slice(20, 35, None))
(slice(15, 0, -1), slice(25, 15, -1))
